# Retrieve climate data through the meteo.cat API

In [3]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt 
import time
import csv
import geojson
import folium
from folium.plugins import HeatMap
import datetime as dt
import dask.dataframe as dd


AttributeError: module 'pandas.core.strings' has no attribute 'StringMethods'

In [26]:
# Check usage
key = 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'

url = 'https://api.meteo.cat/quotes/v1/consum-actual'

headers = {'X-API-KEY': 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'}
requests.get(url, headers=headers).json()

{'client': {'nom': 'Pau Torrente Badia UB'},
 'plans': [{'nom': 'XEMA_750 OD',
   'periode': 'Mensual',
   'maxConsultes': 750,
   'consultesRestants': 702,
   'consultesRealitzades': 48},
  {'nom': 'Predicció_100',
   'periode': 'Mensual',
   'maxConsultes': 100,
   'consultesRestants': 100,
   'consultesRealitzades': 0},
  {'nom': 'XDDE_250',
   'periode': 'Mensual',
   'maxConsultes': 250,
   'consultesRestants': 250,
   'consultesRealitzades': 0},
  {'nom': 'Quota',
   'periode': 'Mensual',
   'maxConsultes': 300,
   'consultesRestants': 300,
   'consultesRealitzades': 0},
  {'nom': 'Referència Bàsic',
   'periode': 'Mensual',
   'maxConsultes': 2000,
   'consultesRestants': 2000,
   'consultesRealitzades': 0}]}

In [ ]:
# Get request to check resource availability  --> Just remove # to run

# url = 'https://api.meteo.cat/quotes/v1/consum-actual'

# headers = {'X-API-KEY': 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'}
# requests.get(url, headers=headers).json()

In [ ]:
# Example of a get request retrieval  --> Just remove # to run

# key = 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'
# url = 'https://api.meteo.cat/xema/v1' + '/estacions/mesurades/X8/2020/03/02'

# headers = {'Accept': 'application/json', 'X-API-KEY': key}

# r = requests.get(url, headers=headers)

# r.json()

In [27]:
# Request example with processing to retrieve rain and temperature for each timestamp and store them as dictionaries on a list  --> Just remove # to run

# key = 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'
# url = 'https://api.meteo.cat/xema/v1' + '/estacions/mesurades/X8/2020/03/02'

# headers = {'Accept': 'application/json', 'X-API-KEY': key}

# r = requests.get(url, headers=headers)

# data = r.json()[0]


# # 35 is for precipitation in mm and 32 for temperature in ºC

# precipitation = [data['variables'][i]['lectures'] for i in range(len(data['variables'])) if data['variables'][i]['codi'] == 35][0]
# temperature = [data['variables'][i]['lectures'] for i in range(len(data['variables'])) if data['variables'][i]['codi'] == 32][0]


# date_variables = [{'timestamp':int(dt.datetime.strptime(d['data'], '%Y-%m-%dT%H:%MZ').timestamp()), 'mm_precip':d['valor']} for d in precipitation]

# for i in range(len(date_variables)):
#     date_variables[i]['temperature'] = temperature[i]['valor']

# date_variables

# Station localization on a map using the folium library

In [2]:

gener_info = pd.read_csv('data/2021_01_Gener_BicingNou_INFORMACIO.csv')

gener_info.head()



,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,nearby_distance,cross_street,last_updated,ttl
0,1,"GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397978,2.180107,16,"GRAN VIA CORTS CATALANES, 760",8013,46,1000,NaN,1609455629,25
1,2,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395488,2.177198,17,"C/ ROGER DE FLOR, 126",8013,27,1000,NaN,1609455629,25
2,3,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394156,2.181331,11,"C/ NÀPOLS, 82",8013,27,1000,NaN,1609455629,25
3,4,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393317,2.181248,8,"C/ RIBES, 13",8013,21,1000,NaN,1609455629,25
4,5,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391103,2.180176,7,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",8018,39,1000,NaN,1609455629,25


In [4]:
gener_estacions = pd.read_csv('data/2021_01_Gener_BicingNou_ESTACIONS.csv')

gener_estacions.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,last_updated,ttl
0,1,43,43,0,3,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22
1,2,17,15,2,10,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22
2,3,3,3,0,22,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22
3,4,15,14,1,6,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22
4,5,8,8,0,31,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22


In [5]:
int(gener_estacions.head()['last_reported'][0])

1609455406

In [9]:
# something like this should be done if we want to account for spain's time dts changes
import pytz
from datetime import datetime

# create a timezone object for the Spanish time zone
spain_tz = pytz.timezone('Europe/Madrid')


og = 1583503600

# create a datetime object representing a timestamp in UTC
utc_time = datetime.utcfromtimestamp(og)

# convert the UTC datetime object to the Spanish time zone
spain_time = utc_time.replace(tzinfo=pytz.utc).astimezone(spain_tz)

# convert the Spanish datetime object to a Unix timestamp
spain_timestamp = int(spain_time.timestamp())

# print the original and corrected timestamps
print("Original timestamp:", og)
print("Corrected timestamp:", spain_timestamp)

Original timestamp: 1583503600
Corrected timestamp: 1583503600


# WEATHER AND STATION DATA MERGE


In [6]:
weather_data = pd.read_csv('weather_data/weather.csv')
weather_data= weather_data.rolling(window=2,  step=1).mean()
weather_data['timestamp'] = weather_data['timestamp'].astype(int)#-1800
weather_data

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [29]:
# weather_data['datetime_formatted'] = 
5//2

2

In [13]:
dataset = pd.merge(left = gener_estacions, right = weather_data, left_on = 'wt_last_reported', right_on = 'timestamp',   how = 'left')

target = dataset['num_bikes_available']
df = dataset['']

In [16]:
import dask.dataframe as dd

data  = dd.read_csv('data/data_bicing_joined_HX.csv', assume_missing=True, delimiter=';')


In [24]:
len = data.shape[0].compute()

In [25]:
len

15209395